# Prediction of Road Sign Direction
### (based on AngleOfSignBoard, Aspect Ratio of clicked image- width/height)

In [1]:
# loading the basic libraries

import pandas as pd
import numpy as np

In [2]:
# loading the dataset

df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# just to check the structure of the numerical values in dataset

df.describe()

,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
count,38485.000000,38485.000000,38485.000000,38485.000000
mean,132.930986,0.901241,92.922957,104.993140
std,98.042472,0.226068,52.399274,53.347424
min,1.000000,0.260000,19.000000,23.000000
25%,42.000000,0.770000,54.000000,65.000000
50%,80.000000,0.930000,80.000000,96.000000
75%,228.000000,1.040000,118.000000,132.000000
max,360.000000,2.790000,589.000000,513.000000


In [4]:
# encoding the mirror directions to 0,1,2,3
def mapping(direction):
    if direction=='Front':return 0
    elif direction=='Right':return 3
    elif direction=='Rear':return 2
    elif direction=='Left':return 1
df['DetectedCamera'] = df['DetectedCamera'].map(mapping)
df['SignFacing (Target)'] = df['SignFacing (Target)'].map(mapping)


In [5]:
# renaming the looongg column names ^_^! 
df.rename(columns={'SignFacing (Target)': 'Actual', 'DetectedCamera': 'Detected'}, inplace=True)

In [6]:
# complete training samples(X) and target values(y)
y = df.pop('Actual')
X = df

In [7]:
# removing the id values(useless)
Id = df.pop('Id')

In [23]:
# splitting data in 70-30 train-test ratio
from sklearn.cross_validation import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state = 42)

In [24]:
# training and testing(log_loss metric) on some classifiers for analysing purpose (RFC performs splendidly)

## tuning of hyperparameter is done here while training and testing analysis on train dataset

### disbaled other two classifiers as were poorly performing on tuning as well

## RFC achieving an awesome 0.106 log_loss value 

from sklearn.ensemble import GradientBoostingClassifier as GBC,RandomForestClassifier as RFC
from sklearn.linear_model import LogisticRegression as LR, RandomizedLogisticRegression as RLR
from sklearn.metrics import log_loss

classifiers =[
    #GBC(learning_rate=0.001, n_estimators=500, max_depth=5, warm_start=True),
    RFC(n_estimators=550, max_features=2, max_depth=10, min_samples_split=8,oob_score=True, warm_start=False),
    #LR(C=0.01, solver='newton-cg', max_iter=300, multi_class='multinomial', warm_start=False),
]

for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    print log_loss(y_test, y_pred)
    

0.105969507399


In [25]:
# refining test data for final submission

test.rename(columns={'DetectedCamera': 'Detected'}, inplace=True)
Id = test.pop('Id')
test['Detected'] = test['Detected'].map(mapping)
test.head()

,Detected,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,3,67,0.63,107,169
1,0,16,0.88,61,69
2,3,44,1.15,232,202
3,3,50,1.10,137,125
4,0,30,0.95,99,104


In [27]:
'''
using complete train file for training for corresponding test file to get probablistic score
for each of 0,1,2,3 classes [Front, Left, Rear, Right]
'''

clf = RFC(n_estimators=550, max_features=2, max_depth=10, min_samples_split=8,oob_score=True, warm_start=False)
clf.fit(X_train,y_train)
y_final = clf.predict_proba(test)


In [28]:
# we can see the probablistic scores here as a numpy array

y_final

array([[  9.98126211e-01,   6.12428707e-05,   6.97055700e-04,
          1.11549027e-03],
       [  9.98247723e-01,   4.60141816e-04,   3.53450890e-05,
          1.25679002e-03],
       [  9.98206249e-01,   9.12645525e-05,   5.94486848e-04,
          1.10799994e-03],
       ..., 
       [  6.02626758e-04,   2.33131666e-03,   9.95801216e-01,
          1.26484082e-03],
       [  7.21814531e-03,   6.43057066e-03,   9.86225221e-01,
          1.26062641e-04],
       [  4.53218624e-04,   2.89134244e-03,   9.95293920e-01,
          1.36151898e-03]])

In [29]:
# saving the submission file 

columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=y_final, columns=columns)
sub.Id = Id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("submission.csv", index=False)